# CLI06 - Query Workers

Query as API (aka Query Workers) enables developers to convert saved C8QL queries into geo-distributed REST APIs quickly. Query Workers eliminate the need for separate backend servers & containers for CRUD operations

## Pre-requisite

Lets assume 
- You have already made a tenant account and have a username and password
- You have installed the Macrometa CLI as explained in section 01
- You have generated an API Key as explained in section 01

In [ ]:
npm install -g gdnsl

## 1. Define Login Credentials


In [ ]:
# Login credentials
url="https://gdn.paas.macrometa.io"
email="email"
api_key="XXXX"
# You can either use LOCAL or ALL also Please enter the name of the regions. For multiple regions enter comma-separated names. For example: region1, region2.
regions="ALL"

## 2. Connect to GDN

Now that you have added your login credentials, lets connect to GDN:

In [ ]:
echo "Creating gdnsl.yaml file" 

echo "url: $url
email: $email
apikey: $api_key
regions:
  - $regions" > gdnsl.yaml

echo "------- CONNECTION SETUP  ------"
# if you are running this from terminal then you can ignore above step and run below command
# gdnsl init

## 3. Create a Geo Replicated Collection

Before we create the Query Workers lets create a collection that we can work with:

In [ ]:
collection_name="person"

echo " ------- CREATE GEO-REPLICATED COLLECTION  ------"
gdnsl collection create $collection_name --type doc 
echo "Created collection: $collection_name"

## 4. Create Query Workers

The Following Code will create a set of Query Worker APIs in GDN - once complete, you can check that they exist from your console access, under Queries, and navigate to “Saved Queries”.

In [ ]:
echo " ------- CREATE RESTQLs  ------"
gdnsl query-worker create insertRecord --value "INSERT {'firstname':@firstname, 'lastname':@lastname, 'email':@email, 'zipcode':@zipcode, '_key': 'key101'} IN $collection_name" # name: insertRecord
gdnsl query-worker create getRecords --value  "FOR doc IN $collection_name RETURN doc" # name: getRecords
gdnsl query-worker create updateRecord --value  "UPDATE 'key101' WITH { \"lastname\": \"cena\" } IN $collection_name" # name: updateRecord
gdnsl query-worker create deleteRecord --value "REMOVE 'key101' IN $collection_name" # name: deleteRecord
gdnsl query-worker create countRecords --value  "RETURN COUNT(FOR doc IN $collection_name RETURN 1)" # name: countRecords
echo " ------- CREATED RESTQLs  ------"

## 5. Execute Query Workers

> Note: We will skip testing the `deleteRecord` API in this section so that we can demonstrate how to update a Query Worker API first.

### 5.1. Test the "insertRecord" Query Worker

In [ ]:
echo " ------- EXECUTE RESTQLs ------"
echo "Insert data...."
gdnsl query-worker run insertRecord --param  "firstname=john" --param "lastname=doe" --param "email=john.doe@macrometa.io" --param "zipcode=511037"
    
echo "Get data...."
gdnsl query-worker run getRecords

### 5.2. Test the "updateRecord" Query Worker

In [ ]:
echo "Update data...."
gdnsl query-worker run updateRecord
echo "Record updated"

### 5.3. Test the "getRecords" Query Worker

In [ ]:
echo "Get data...."
gdnsl query-worker run getRecords

### 5.4. Test the "countRecords" Query Worker

In [ ]:
echo "Count records...."
gdnsl query-worker run countRecords

## 6. Updating a Query Worker API

### 6.1. Test that we can query GDN and see the document

A common requirement when building an application is to make a change to an API and update it. 

To demonstrate this, let's first query the database using the `execute_query` method we learned in an earlier section

Once run, we should see the document in the cell output in JSON format so we can read it easily.

In [ ]:
echo "Get docs in \"person\" collection"

gdnsl query "FOR doc IN person RETURN doc"

### 6.2. Updating a Query Worker API

The following cell shows us how to add a new parameter to the Query. In this example, we will add the parameter "phone" to the query.

In [ ]:
gdnsl query-worker update insertRecord --value "INSERT {'firstname':@firstname, 'lastname':@lastname, 'email':@email, 'zipcode':@zipcode, 'phone': @phone} IN $collection_name"

### 6.4. Testing the updated Query Worker API

Now that we have updated the query, let's add a new record using it to test that it works.

In [ ]:
echo "Insert data with updated query worker"
gdnsl query-worker run insertRecord --param "firstname=john" --param "lastname=doe" --param "email=john.doe@macrometa.io" --param "zipcode=511037" --param "phone=213-555-9578"

### 6.5. Checking the data in the collection

Now, let's check that the data has been updated in the record by using the `execute_query` method - we should see two documents. 

1. the original document without a phone number, and
2. a second document, but now with a phone number added

> Hint: If you would like to return to the step “countRecords” and re-run the code, the count value should increase by 1.

In [ ]:
echo "Get docs in \"person\" collection, after executing the updated query"

gdnsl query "FOR doc IN person RETURN doc"

## 7.  Deleting a record using a Query Worker API

In [ ]:
echo "Delete data...."
gdnsl query-worker run deleteRecord

## 8. Delete Query Workers

We might want to delete our Query Worker APIs completely.

For each Query Worker we want to remove, we use the `delete_restql` method as shown in the cell below:

In [ ]:
echo "------- DELETE RESTQLs ------"
gdnsl query-worker delete insertRecord
gdnsl query-worker delete getRecords
gdnsl query-worker delete updateRecord
gdnsl query-worker delete countRecords
gdnsl query-worker delete deleteRecord
echo "------- DONE  ------"

## 9. Delete the Collection

Now let us remove the collection we added during this tutorial:

In [ ]:
gdnsl collection delete $collection_name

## Section Completed!

Congratulations! You have completed this tutorial.

**Note: To use the created Query Worker APIs you should use the API reference documentation for detailed instructions.**

e.g. `/_fabric/{fabric}/_api/restql/{name}`

Where `{name}` is the name of your Query Worker API, e.g. "insertRecord".